In [1]:
# Using dependencies:
# FSL version: 5.0.11

# Nipype 1.1.4
# installed 2018-11-01 with the command:
# pip3 install nipype[all]

# Shold try: https://github.com/kaczmarj/neurodocker

In [50]:
# Uncovering all NIFTI files and folder structures to be corrected
import os

# GE_blip_nii_pairs and SE_blip_nii_pairs
# have the following structure 
# [('NIFTI/path/to/negative_phase_encoding_direction.nii', 'NIFTI/path/to/positive_phase_encoding_direction.nii'), (,) , .. ]

# assuming e1.nii is GE and e2.nii is SE

# assuming prescan corresponds to corr_SENSE
# and scan corresponds to SENSE

# assuming prescan and corr_SENSE are always negative phase encoding directions
# and scan and SENSE positive phase encoding directions

# Empty lists for relative directory path + filename.nii to be corrected

GE_prescan = []
GE_scan = []

GE_corr_SENSE = []
GE_SENSE = []

GE_not_corr = []
GE_corr = []

SE_prescan =[]
SE_scan = []

SE_corr_SENSE = []
SE_SENSE = []

SE_not_corr = []
SE_corr = []

num_directories_bypassed = 0

# NIFTI directory traversal. The NIFTI directory 
# is the output of dicom_to_niix_same_folder_structure.sh
# conversion script (from DICOM files) using dcm2niix 1.0.20180622
NIFTI_folder_name = "NIFTI"
directoryTree = [tuple3 for tuple3 in os.walk(NIFTI_folder_name)]


#for dirpath, dirnames, filenames in directoryTree:
    
for i in range(len(directoryTree)):
    
    dirpath, dirnames, filenames = directoryTree[i]
    
    # Start, two available folders check
    
    # os.walk iterates top-down as default. 
    
    # With the assumed folder structure
    # it visits the folders according to this example:
    
    # Iteration 1: Show EPI-blip-down-folder and EPI-blip-up-folder
    # Iteration 2: Visit either EPI-blip-down-folder or EPI-blip-up-folder
    # to show its contents.
    
    # This means that os.walk will always visit the directory
    # with the blip-up and blip-down folders in ONE ITERATION
    # BEFORE visiting one of the folders.
    
    # The two available folders check enshures that only
    # directories corresponding to Iteration 1 in the 
    # example with exactly two folders are visited
    # by the script. When there exist two folders,
    # they are assumed to be the blip-down and blip-up 
    # folders (this should be assured if they are the only folders)
    # with the string "epi" as part of lowercase versions of folder names
    # in the original DICOM directory structure. 
    # If this is assured, dicom_to_niix_same_folder_structure.sh
    # will only convert those folders to .nii format into a near equal
    # folder structure. The difference is that the relative path from
    # root of script starts with the folder NIFTI instead of the 
    # orignal folder DICOM containing the original .dcm folder structure.
    
    # A single folder shown when os.walk is
    # in the iteration analog to Iteration 1 where
    # "epi" is part of the the name (incase-sesitive), is bypassed
    # from furhter subdirectory traversal.
    # This is done by skipping the newxt iteration that would have followed, 
    # where the .nii files for the single epi folder would 
    # have been accessed/shown by os.walk.
    
    # Assume that the two available folders check
    # enshures that only .nii files associated with
    # two different EPI phase encoding directions
    # are included for corretion.
    
    if i > 0:
        prevdirpath, prevdirnames, _ = directoryTree[i-1]
        if len(prevdirnames) != 2 and prevdirnames and \
        "epi" in prevdirnames[0].lower():
            print("WARNING: The folder " + prevdirnames[0] + \
                  " is the only folder in the directory")
            print(prevdirpath)
            print("Assuming it does not have a companying folder \
            with opposite phase endoded EPI recording.")
            print("Conclusion: Skipping data in " + prevdirnames[0])
            num_directories_bypassed += 1
            continue

    # End, two available folders check 

    # Enshure not empty list traversal
    if filenames:

        for file in filenames:
            
            # Enshure only .nii files
            if file[-len(".nii"):] == ".nii":
                
                
                # Gradient Echo files    

                if determine_e1_or_e2(file) == "e1" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "prescan":
                    # GE negative phase encoded
                    GE_prescan += [os.path.join(dirpath, file)]

                elif determine_e1_or_e2(file) == "e1" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "scan":
                    # GE positive phase encoded
                    GE_scan += [os.path.join(dirpath, file)]

                elif determine_e1_or_e2(file) == "e1" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "corr_SENSE":
                    # GE negative phase encoded
                    GE_corr_SENSE += [os.path.join(dirpath, file)]

                elif determine_e1_or_e2(file) == "e1" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "SENSE":
                    # GE positive phase encoded
                    GE_SENSE += [os.path.join(dirpath, file)]
                    
                # the two last are for a special case
                # with a EPI pair recording
                # where corr seems to be positive phase
                # encoded and the other file name has nothing
                # in it's name that is specified and seems to
                # be negative phase encoded recording.
                elif determine_e1_or_e2(file) == "e1" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "not_corr":
                    # GE negative phase encoded
                    GE_not_corr += [os.path.join(dirpath, file)]

                elif determine_e1_or_e2(file) == "e1" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "corr":
                    # GE positive phase encoded
                    GE_corr += [os.path.join(dirpath, file)]

                    
                # Spin Echo files

                elif determine_e1_or_e2(file) == "e2" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "prescan":
                    # SE negative phase encoded
                    SE_prescan += [os.path.join(dirpath, file)]

                elif determine_e1_or_e2(file) == "e2" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "scan":
                    # SE positive phase encoded
                    SE_scan += [os.path.join(dirpath, file)]

                elif determine_e1_or_e2(file) == "e2" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "corr_SENSE":
                    # SE negative phase encoded
                    SE_corr_SENSE += [os.path.join(dirpath, file)]

                elif determine_e1_or_e2(file) == "e2" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "SENSE":
                    # SE positive phase encoded
                    SE_SENSE += [os.path.join(dirpath, file)]

                # the two last are for a special case
                # with a EPI pair recording
                # where corr seems to be positive phase
                # encoded and the other file name has nothing
                # in it's name that is specified and seems to
                # be negative phase encoded recording.                    
                elif determine_e1_or_e2(file) == "e2" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "not_corr":
                    # negative phase encoded
                    SE_not_corr += [os.path.join(dirpath, file)]

                elif determine_e1_or_e2(file) == "e2" and \
                determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file) == "corr":
                    # positive phase encoded
                    SE_corr += [os.path.join(dirpath, file)]

                    
print("----------------------------------------------------------------")

            
print("1. Equal number of GE and SE prescan and scan included: ", end="")
print(len(GE_prescan) == len(GE_scan) == len(SE_prescan) == len(SE_scan))
print("2. Equal number of GE and SE corr_SENSE and SENSE included: ", end="")
print(len(GE_corr_SENSE) == len(GE_SENSE) == len(SE_corr_SENSE) == len(SE_SENSE))
print("3. Equal number of GE and SE not_corr and corr included: ", end="")
print(len(GE_not_corr) == len(GE_corr) == len(SE_not_corr) == len(SE_corr))


# Finally, make GE and SE list of 2D tuples of relative path + filename.nii
# for blip-down - blip-up EPI pairs. The two lists is are used later for topup and EPIC.
GE_blip_nii_pairs = list(zip(GE_prescan, GE_scan)) + \
    list(zip(GE_corr_SENSE, GE_SENSE)) + list(zip(GE_not_corr, GE_corr))
SE_blip_nii_pairs = list(zip(SE_prescan, SE_scan)) + \
    list(zip(SE_corr_SENSE, SE_SENSE)) + list(zip(SE_not_corr, SE_corr))

print("4. Equal number of GE and SE prescan+corr_SENSE+not_corr and scan+SENSE+corr included: ", end="")
print(len(GE_blip_nii_pairs) == len(SE_blip_nii_pairs))

print("----------------------------------------------------------------")

print("Important assumptions:")
print("* Assuming the following content of a NIFTI file")
print("according to the following string as part of its file name:")
print("e1.nii is GE and e2.nii is SE")
print("* Assuming file and directory names including the string prescan")
print("correspond to file and directory names including the string corr_SENSE")
print("* Assuming file and directory names including the string scan")
print("correspond to file and directory names including the string SENSE")
print("* Assuming file and directory names including no special string")
print("but is placed correctly according to the directory to compare to")
print("correspond to file and directory names including the string corr")
print("* 1., 2. and 3. and 4. must be True")
print("* prescan, corr_SENSE and not_corr: Negative phase encoded direction EPI (blip-down)")
print("* scan, SENSE and corr: Positive phase encoded direction EPI (blip-up)")
print("* prescan and corr_SENSE EPI have generally fewer number of temporal \
recordings than scan and SENSE EPI")

print("----------------------------------------------------------------")

print("Report when all assumptions are fullfilled:")
print("* Number of GE and SE prescan-scan pairs found: %i" % len(GE_prescan))
print("* Number of GE and SE corr_SENSE-SENSE pairs found: %i" % len(GE_corr_SENSE))
print("* Number of GE and SE not_corr-corr pairs found: %i" % len(GE_not_corr))
print("* Total number of GE and SE blip-down - blip-up pairs: %i" % len(GE_blip_nii_pairs))
print("* Total number of GE and SE blip-down - blip-up pairs: %i + %i = %i" % \
      (len(GE_prescan), len(GE_corr_SENSE), len(GE_prescan) + len(GE_corr_SENSE) + len(GE_not_corr)) )
print("* Number of directories bypassed since no detection of opposite phase encoded EPI pairs: %i" % \
      num_directories_bypassed)
print("Sanity check: %i should be the original number of EPI blip-down-blip-up folders \
inside the input directy to this script, with directory root folder name: %s. \
This should be the output directory of dicom_to_niix_same_folder_structure.sh beforehand." % \
      (len(GE_blip_nii_pairs) + num_directories_bypassed, NIFTI_folder_name))

NameError: name 'determine_e1_or_e2' is not defined

In [6]:
def remove_substring_after_last_slash(string_with_slashes):
    # Index for last "/" in string_with_slashes
    last_slash_index = len(string_with_slashes) - 1 - string_with_slashes[::-1].index("/")
    
    # Chars after last "/" are removed
    return string_with_slashes[:last_slash_index]

def longest_common_substring_from_beginning(string1, string2):
    if string1[0] != string2[0]:
        return ""
    
    if len(string1) < len(string2):
        index_differ = [i for i in range(len(string1)) if string1[:i] == string2[:i]][-1]
        longest_common_substring_from_beginning =  string1[:index_differ]
        
    else:
        index_differ = [i for i in range(len(string2)) if string2[:i] == string1[:i]][-1]
        longest_common_substring_from_beginning =  string2[:index_differ]
    return longest_common_substring_from_beginning

def find_common_folder_for_blip_down_blip_up_EPI_data(blip_down_file, blip_up_file):
    # Example:
    # if blip_down_file:
    # 'NIFTI/Anonymized/DEFACED_IVS/1269262582/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan/142048_GE-SE_EPI_SSH_v1_32CH_V2_prescan_901_e1.nii'
    # and blip_down_file:
    # 'NIFTI/Anonymized/DEFACED_IVS/1269262582/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_scan/142048_GE-SE_EPI_SSH_v1_32CH_V2_scan_1001_e1.nii'
    # Shall return:
    # 'NIFTI/Anonymized/DEFACED_IVS/1269262582/DAY_0000'
    common_path = longest_common_substring_from_beginning(blip_down_file, blip_up_file)
    return remove_substring_after_last_slash(common_path)

def extract_file_name(path_and_file_name):
    # Finds the first index of "/" in the reversed
    # order string or path_and_file_name.
    # Then converts the reversed index to normal index.
    i = len(path_and_file_name) - path_and_file_name[::-1].index("/")
    # File name is the last part of path_and_file_name
    # after index i
    return path_and_file_name[i:]

def determine_e1_or_e2(file_name):
    if file_name[-len("e1.nii"):] == "e1.nii":
        # e1 is GE
        return "e1"
    elif file_name[-len("e1.nii"):] == "e2.nii":
        # e2 is SE
        return "e2"
    else:
        print("PID %i: Error, could not determine e1 (GE) or e2 (SE) \
        from file_name, aborting process" % os.getpid())
        sys.exit(1)

def determine_prescan_or_scan_or_corr_SENSE_or_SENSE(file_name):
    if "prescan" in file_name.lower():
        return "prescan"
    elif "scan" in file_name.lower():
        return "scan"
    elif "corr_sense" in file_name.lower():
        return "corr_SENSE"
    elif "sense" in file_name.lower():
        return "SENSE"

    # the two lasts elifs
    # are special cases.
    # Should be removed
    # for clean data
    
    elif "corr" in file_name.lower():
        # expected positive 
        # phase encoded
        return "corr"

    elif ".nii" in file_name.lower():
        # expected negative
        # phase encoded
        return "not_corr"

    else:
        print("PID %i: Error, could not determine prescan or scan or \
        corr_SENSE or SENSE from file_name, aborting process" % os.getpid())
        print(file_name)
        sys.exit(1)
    
def determine_merged_blips_file_name(blip_down_file_name, \
                                     blip_up_file_name, \
                                    temporary_window_number="0000"):
    # The returned name must correspond to the naming scheme
    # followed by the use of fslsplit in the script.
    
    if not determine_e1_or_e2(blip_down_file_name) == \
    determine_e1_or_e2(blip_up_file_name):
        print("PID %i: Error, the arguments blip_down_file_name and \
        blip_up_file_name in the determine_merged_blips_file_name \
        function coud not be identified with the same echo sequence \
        (e1 (GE) or e2 (SE)), aborting process" % os.getpid())
        sys.exit(1)
    
    blip_down_type = determine_prescan_or_scan_or_corr_SENSE_or_SENSE(blip_down_file_name)
    blip_up_type = determine_prescan_or_scan_or_corr_SENSE_or_SENSE(blip_up_file_name)
    echo_type = determine_e1_or_e2(blip_down_file_name)

    blip_down_blip_up_longest_common_substring_from_beginning = \
        longest_common_substring_from_beginning(blip_down_file_name, \
                                                blip_up_file_name)

    blip_down_number_before_echo_type_in_name = \
        blip_down_file_name[len(blip_down_blip_up_longest_common_substring_from_beginning + \
           blip_down_type) + 1:][:-(len(echo_type + ".nii") + 1)]

    blip_up_number_before_echo_type_in_name = \
        blip_up_file_name[len(blip_down_blip_up_longest_common_substring_from_beginning + \
           blip_up_type) + 1:][:-(len(echo_type + ".nii") + 1)]

    merged_blips_file_name = \
        blip_down_blip_up_longest_common_substring_from_beginning + \
        blip_down_type + "_" + \
        blip_up_type + "_" + \
        blip_down_number_before_echo_type_in_name + "_" + \
        blip_up_number_before_echo_type_in_name + "_" + \
        determine_e1_or_e2(blip_down_file_name) + "_" + \
        temporary_window_number + ".nii"
    
    return merged_blips_file_name

blip_down_file_name = extract_file_name(blip_down_file)
blip_up_file_name = extract_file_name(blip_up_file)

print(blip_down_file_name)
print(blip_up_file_name)

blip_down_blip_up_temporary_window_file_name = determine_merged_blips_file_name(blip_down_file_name, \
                                                               blip_up_file_name)
print(blip_down_blip_up_temporary_window_file_name)

blip_down_blip_up_temporary_window_file = output_path + "/" + blip_down_blip_up_temporary_window_file_name

print(blip_down_blip_up_temporary_window_file)

def determine_blip_file_name_for_window(blip_direction , \
    blip_file_name, temporary_window_number="0000"):
    
    process_msg_prefix = "PID %i: " % os.getpid()
    
    if blip_direction == "blip_down":    
        return blip_down_file_name[:-len(determine_e1_or_e2(blip_down_file_name) + ".nii")] + \
        determine_e1_or_e2(blip_down_file_name) + "_" + \
        temporary_window_number + ".nii"
    
    elif blip_direction == "blip_up":
        return blip_up_file_name[:-len(determine_e1_or_e2(blip_up_file_name) + ".nii")] + \
        determine_e1_or_e2(blip_up_file_name) + "_" + \
        temporary_window_number + ".nii"
    
    else:
        print(process_msg_prefix + " Error: determine_blip_file_name_for_window: \
        the argument blip_direction is not correctly set")
        sys.exit(1)
        
blip_down_temporary_window_file_name = determine_blip_file_name_for_window("blip_down", blip_down_file_name)
blip_down_temporary_window_file = output_path + "/" + blip_down_temporary_window_file_name

print(blip_down_temporary_window_file)

blip_up_temporary_window_file_name = determine_blip_file_name_for_window("blip_up", blip_up_file_name)
blip_up_temporary_window_file = output_path + "/" + blip_up_temporary_window_file_name

print(blip_up_temporary_window_file)

94900_GE-SE_EPI_SSH_v1_32CH_V2_prescan_901_e1.nii
94900_GE-SE_EPI_SSH_v1_32CH_V2_scan_1001_e1.nii
94900_GE-SE_EPI_SSH_v1_32CH_V2_prescan_scan_901_1001_e1_0000.nii
TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_prescan_scan_901_1001_e1_0000.nii
TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_prescan_901_e1_0000.nii
TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_scan_1001_e1_0000.nii


In [7]:
blip_down_file_name

'94900_GE-SE_EPI_SSH_v1_32CH_V2_prescan_901_e1.nii'

In [5]:
import sys

blip_down_file = 'NIFTI/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan/94900_GE-SE_EPI_SSH_v1_32CH_V2_prescan_901_e1.nii'
blip_up_file = 'NIFTI/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_scan_1001_e1.nii'

#def topup(blip_down_file, blip_up_file):
    #common_data_dir = find_common_folder_for_blip_down_blip_up_EPI_data(blip_down_file, blip_up_file)
    
common_dir = find_common_folder_for_blip_down_blip_up_EPI_data(blip_down_file, \
                                                               blip_up_file)

process_msg_prefix = "PID %i: " % os.getpid()

# Enshure that the input is correct 1:
# Asumption: The mother folder a given scan collection has always "day"
# in it's lowercase version of folder name.
if not "day" in common_dir.lower():
    print(process_msg_prefix + " Error: it seems that common_dir is wrong: \
    " + common_dir + " , aborting process")
    sys.exit(1)

TOPUP_folder_name = "TOPUP"

# Enshure that the input is correct 2:
# Enshure that echo sequences match between the files (GE or SE)
if not determine_e1_or_e2(blip_down_file_name) == \
    determine_e1_or_e2(blip_up_file_name):
    print(process_msg_prefix + " Error: Could not verify \
    matching echo sequence EPI pair (GE or SE), aborting process")
    sys.exit(1)

    
output_path = TOPUP_folder_name + \
    longest_common_substring_from_beginning(blip_down_file, \
                                            blip_up_file)[len(NIFTI_folder_name):] + \
    determine_prescan_or_scan_or_corr_SENSE_or_SENSE(blip_down_file_name) + "_" + \
    determine_prescan_or_scan_or_corr_SENSE_or_SENSE(blip_up_file_name)

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [896]:
# Cell is not used yet.

# This function is not used until the script is updated
# with a method to add two empty slices, each on top and bottom
# along the z axis of a NIFTI volume.
def determine_merged_blip_and_emtpy_slices_file_name(blip_direction, \
                                                     blip_down_file_name, \
                                                     blip_up_file_name):
    if blip_direction == "blip_down":
        return longest_common_substring_from_beginning(blip_down_file_name, \
                                                       blip_up_file_name) + \
        "empty_slices_added_" + \
        determine_prescan_or_scan_or_corr_SENSE_or_SENSE(blip_down_file_name) + \
        "_" + determine_e1_or_e2(blip_down_file_name) + ".nii"
    
    elif blip_direction == "blip_up":
        return longest_common_substring_from_beginning(blip_down_file_name, \
                                                       blip_up_file_name) + \
        "empty_slices_added_" + \
        determine_prescan_or_scan_or_corr_SENSE_or_SENSE(blip_up_file_name) + \
        "_" + determine_e1_or_e2(blip_up_file_name) + ".nii"

    else:
        print("PID %i: Error in function determine_merged_blip_and_emtpy_slices_file_name. \
        blip_direction not correctly specified [blip_down, blip_up], aborting process" % os.getpid())
        sys.exit(1)      

blip_down_file_name_empty_slices_added = \
    determine_merged_blip_and_emtpy_slices_file_name("blip_down", blip_down_file_name, blip_up_file_name)

print(blip_down_file_name_empty_slices_added)
    
blip_up_file_name_empty_slices_added = \
    determine_merged_blip_and_emtpy_slices_file_name("blip_up", blip_down_file_name, blip_up_file_name)

print(blip_up_file_name_empty_slices_added)

print("-------------------------------------------")

blip_down_file_empty_slices_added = output_path + "/" + \
    blip_down_file_name_empty_slices_added

print(blip_down_file_empty_slices_added)

print("-------------------------------------------")

blip_up_file_empty_slices_added = output_path + "/" + \
    blip_up_file_name_empty_slices_added

print(blip_up_file_empty_slices_added)

94900_GE-SE_EPI_SSH_v1_32CH_V2_empty_slices_added_prescan_e1.nii
94900_GE-SE_EPI_SSH_v1_32CH_V2_empty_slices_added_scan_e1.nii
-------------------------------------------
TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_empty_slices_added_prescan_e1.nii
-------------------------------------------
TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_empty_slices_added_scan_e1.nii


In [897]:
from subprocess import call

def run_shell_command(full_command):
    process_msg_prefix = "PID %i: " % os.getpid()
    try:
        retcode = call(full_command, shell=True)
        if retcode < 0:
            print(process_msg_prefix + "Child was \
            terminated by signal", -retcode, file=sys.stderr)
        #else:
        #    print(process_msg_prefix + "fslsplit: Child returned", retcode, file=sys.stderr)
    except OSError as e:
        print(process_msg_prefix + "Error: Child execution \
        failed:", e, file=sys.stderr)

# Splitting NIFTI files along time axis and moving to destination for correction

def split_NIFTI_file_along_time_axis_and_move(output_path, \
                                              blip_file, blip_file_name):
    # blip_file is the relative path from script root + file name
    # blip_file_name is the file name only
    
    process_msg_prefix = "PID %i: " % os.getpid()
    
    output_base_name = output_path + "/" + blip_file_name[:-len(".nii")] + "_"

    pre_command = 'FSLOUTPUTTYPE=NIFTI'
    command = 'fslsplit ' + '"' + blip_file + '"' + ' ' + '"' + \
        output_base_name + '"' + ' -t'
    full_command = pre_command + ' && ' + command

    run_shell_command(full_command)
    
    print(process_msg_prefix + "Successfully split " + \
                  blip_file + " into directory " + output_path + \
                 " using fslsplit in subprocess shell call")
        
split_NIFTI_file_along_time_axis_and_move(output_path, \
                                          blip_down_file, \
                                          blip_down_file_name)
split_NIFTI_file_along_time_axis_and_move(output_path, \
                                          blip_up_file, \
                                          blip_up_file_name)

PID 5557: Successfully split NIFTI/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan/94900_GE-SE_EPI_SSH_v1_32CH_V2_prescan_901_e1.nii into directory TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan using fslsplit in subprocess shell call
PID 5557: Successfully split NIFTI/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_scan_1001_e1.nii into directory TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan using fslsplit in subprocess shell call


In [898]:
print(blip_down_blip_up_temporary_window_file)
print(blip_down_temporary_window_file)
print(blip_up_temporary_window_file)

TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_prescan_scan_901_1001_e1_0000.nii
TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_prescan_901_e1_0000.nii
TOPUP/Anonymized/DEFACED_IVS/274299819/DAY_0000/No_DeFacing_GE-SE EPI SSH_v1_32CH_V2_prescan_scan/94900_GE-SE_EPI_SSH_v1_32CH_V2_scan_1001_e1_0000.nii


In [280]:
# fslmerge

# Discarding adding empty top and bottom slice along z-axis for now.
# Future fix.

def merge_blip_down_blip_up_first_temporary_window(blip_down_blip_up_temporary_window_file, \
                                                  blip_down_temporary_window_file, \
                                                  blip_up_temporary_window_file):
    # Merge nii files of different phase 
    
    # Assuming that fslsplit correctly appended
    # 0000 for the first temporary window 
    # at the end of the file name
    
    # Assuming that corresponding data
    # for both blip directions already
    # exist in output_path
    
    pre_command = 'FSLOUTPUTTYPE=NIFTI'
    command = 'fslmerge -t ' + '"' + blip_down_blip_up_temporary_window_file + \
        '"' + ' ' + '"' + blip_down_temporary_window_file + \
        '"' + ' ' + '"' + blip_up_temporary_window_file + '"'
    full_command = pre_command + ' && ' + command
    
    run_shell_command(full_command)
    
    print(process_msg_prefix + "Successfully merged " + \
              blip_down_temporary_window_file + " with " + \
              blip_up_temporary_window_file + " into the file " + \
              blip_down_blip_up_temporary_window_file + \
              " using fslmerge in subprocess shell call")

merge_blip_down_blip_up_first_temporary_window(blip_down_blip_up_temporary_window_file, \
                                                  blip_down_temporary_window_file, \
                                                  blip_up_temporary_window_file)

NameError: name 'blip_down_blip_up_temporary_window_file' is not defined

In [ ]:
# TOPUP

from nipype.interfaces.fsl import TOPUP

topup = TOPUP()
topup.inputs.in_file = "b0_b0rev.nii"



topup.inputs.encoding_file = "aquisition_parameters.txt"
topup.inputs.output_type = "NIFTI_GZ"
#topup.cmdline
#'topup --config=b02b0.cnf --datain=aquisition_parameters.txt --imain=b0_b0rev.nii --out=b0_b0rev_base --iout=b0_b0rev_corrected.nii.gz --fout=b0_b0rev_field.nii.gz --jacout=jac --logout=b0_b0rev_topup.log --rbmout=xfm --dfout=warpfield'
res = topup.run()


In [ ]:
"""
# ApplyTOPUP


from nipype.interfaces.fsl import ApplyTOPUP
applytopup = ApplyTOPUP()
applytopup.inputs.in_files = ["epi.nii", "epi_rev.nii"]
applytopup.inputs.encoding_file = "topup_encoding.txt"
applytopup.inputs.in_topup_fieldcoef = "topup_fieldcoef.nii.gz"
applytopup.inputs.in_topup_movpar = "topup_movpar.txt"
applytopup.inputs.output_type = "NIFTI_GZ"
applytopup.cmdline
'applytopup --datain=topup_encoding.txt --imain=epi.nii,epi_rev.nii --inindex=1,2 --topup=topup --out=epi_corrected.nii.gz'
res = applytopup.run() 

"""

In [511]:
"""
Running all corrections

from multiprocessing import Pool

def EPI_correction(blip_down_file, blip_up_file):
    #process_msg_prefix = "PID %i: " % os.getpid()
    #print(process_msg_prefix + "blip down file: " + blip_down_file)
    #print(process_msg_prefix + "blip up file: " + blip_up_file)
    
    #common_data_dir = find_common_folder_for_blip_down_blip_up_EPI_data(blip_down_file, blip_up_file)
    
    #print(process_msg_prefix + "Finished " + common_data_dir)
    
    return 0

p = Pool(processes=1, maxtasksperchild=1)

_ = p.starmap(EPI_correction, GE_blip_nii_pairs + SE_blip_nii_pairs)
"""

In [ ]:
f = open(report_file, 'a+') 
f.flu